# Managed Tables

Before creating table we need to create schema or database and then we create table in it.

In [0]:
%sql
CREATE SCHEMA Infosys;

In [0]:
%sql
CREATE TABLE infosys.employees(
 emp_id INT,
 full_name VARCHAR(30),
 join_date DATE,
 salary INT
);

In [0]:
%sql
INSERT INTO infosys.employees (emp_id, full_name, join_date, salary)
VALUES 
(1, 'Alice Johnson', '2022-01-15', 70000),
(2, 'Bob Smith', '2021-08-01', 80000),
(3, 'Charlie Lee', '2023-03-10', 65000),
(4, 'Diana Prince', '2020-11-25', 90000); 

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
SELECT * FROM infosys.employees;

emp_id,full_name,join_date,salary
1,Alice Johnson,2022-01-15,70000
2,Bob Smith,2021-08-01,80000
3,Charlie Lee,2023-03-10,65000
4,Diana Prince,2020-11-25,90000


In [0]:
%sql
DESCRIBE EXTENDED infosys.employees;

col_name,data_type,comment
emp_id,int,null
full_name,varchar(30),null
join_date,date,null
salary,int,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,infosys,
Table,employees,
Created Time,Sun May 11 09:29:03 UTC 2025,


In [0]:
dbutils.fs.ls("dbfs:/user/hive/warehouse/infosys.db/employees")

Out[8]: [FileInfo(path='dbfs:/user/hive/warehouse/infosys.db/employees/_delta_log/', name='_delta_log/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/user/hive/warehouse/infosys.db/employees/part-00000-d1e73718-2199-4666-bee8-7cfa3dbfe75d-c000.snappy.parquet', name='part-00000-d1e73718-2199-4666-bee8-7cfa3dbfe75d-c000.snappy.parquet', size=1467, modificationTime=1746955756000)]

# External Tables

For creating external table we need to specifiy the location where to be stored while creating the table

In [0]:
%sql
CREATE TABLE infosys.employee_ext(
  emp_id INT,
  full_name VARCHAR(30),
  join_date DATE,
  salary INT
) LOCATION 'dbfs:/tmp/external_employee_table/';

In [0]:
%sql
INSERT INTO infosys.employee_ext (emp_id, full_name, join_date, salary)
VALUES 
(1, 'Alice Johnson', '2022-01-15', 70000),
(2, 'Bob Smith', '2021-08-01', 80000),
(3, 'Charlie Lee', '2023-03-10', 65000),
(4, 'Diana Prince', '2020-11-25', 90000); 

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
SELECT * FROM infosys.employee_ext;

emp_id,full_name,join_date,salary
1,Alice Johnson,2022-01-15,70000
2,Bob Smith,2021-08-01,80000
3,Charlie Lee,2023-03-10,65000
4,Diana Prince,2020-11-25,90000


In [0]:
%sql
DESCRIBE EXTENDED infosys.employee_ext;

col_name,data_type,comment
emp_id,int,null
full_name,varchar(30),null
join_date,date,null
salary,int,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,infosys,
Table,employee_ext,
Created Time,Sun May 11 09:41:29 UTC 2025,


In [0]:
dbutils.fs.ls('dbfs:/tmp/external_employee_table')

Out[15]: [FileInfo(path='dbfs:/tmp/external_employee_table/_delta_log/', name='_delta_log/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/tmp/external_employee_table/part-00000-c12430f3-5f0e-41fa-b819-86d4cf03ec86-c000.snappy.parquet', name='part-00000-c12430f3-5f0e-41fa-b819-86d4cf03ec86-c000.snappy.parquet', size=1467, modificationTime=1746956560000)]

# Difference between managed table and external table

If we delete managed tables the data which is stored also will be deleted where as in external table data still exists if table is dropped.
if we create same table in sample location then again the old data can be used.

In [0]:
%sql
DROP TABLE infosys.employees;
DROP TABLE infosys.employee_ext;

In [0]:
%sql
SELECT * FROM infosys.employees;
SELECT * FROM infosys.employee_ext;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3691950551348865>:8
      6     display(df)
      7     return df
----> 8   _sqldf = ____databricks_percent_sql()
      9 finally:
     10   del ____databricks_percent_sql

File <command-3691950551348865>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   spark.sql(base64.standard_b64decode("U0VMRUNUICogRlJPTSBpbmZvc3lzLmVtcGxveWVlcw==").decode())
      5   df = spark.sql(base64.standard_b64decode("U0VMRUNUICogRlJPTSBpbmZvc3lzLmVtcGxveWVlX2V4dA==").decode())
      6   display(df)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, functio

In [0]:
dbutils.fs.ls("dbfs:/user/hive/warehouse/infosys.db/employees")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-3691950551348867>:1
----> 1 dbutils.fs.ls("dbfs:/user/hive/warehouse/infosys.db/employees")

File /databricks/python_shell/dbruntime/dbutils.py:364, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    362 exc.__context__ = None
    363 exc.__cause__ = None
--> 364 raise exc

ExecutionError: An error occurred while calling o554.ls.
: java.io.FileNotFoundException: /user/hive/warehouse/infosys.db/employees
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:121)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:69)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.listStatus(DatabricksFileSystemV1.scala:179)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(Data

In [0]:
dbutils.fs.ls('dbfs:/tmp/external_employee_table')

Out[19]: [FileInfo(path='dbfs:/tmp/external_employee_table/_delta_log/', name='_delta_log/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/tmp/external_employee_table/part-00000-c12430f3-5f0e-41fa-b819-86d4cf03ec86-c000.snappy.parquet', name='part-00000-c12430f3-5f0e-41fa-b819-86d4cf03ec86-c000.snappy.parquet', size=1467, modificationTime=1746956560000)]

In [0]:
%sql
CREATE TABLE infosys.employee_ext(
  emp_id INT,
  full_name VARCHAR(30),
  join_date DATE,
  salary INT
) LOCATION 'dbfs:/tmp/external_employee_table/';

In [0]:
%sql
INSERT INTO infosys.employee_ext (emp_id, full_name, join_date, salary)
VALUES 
(1, 'Alice Johnson', '2022-01-15', 70000),
(2, 'Bob Smith', '2021-08-01', 80000),
(3, 'Charlie Lee', '2023-03-10', 65000),
(4, 'Diana Prince', '2020-11-25', 90000); 

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
SELECT * FROM infosys.employee_ext;

emp_id,full_name,join_date,salary
1,Alice Johnson,2022-01-15,70000
2,Bob Smith,2021-08-01,80000
3,Charlie Lee,2023-03-10,65000
4,Diana Prince,2020-11-25,90000
1,Alice Johnson,2022-01-15,70000
2,Bob Smith,2021-08-01,80000
3,Charlie Lee,2023-03-10,65000
4,Diana Prince,2020-11-25,90000
